# 🔬 HEXIF Core Viewer

Interactive visualization for H&E → ORION predictions.

**Usage:** Run all cells, then use the interactive widgets to explore cores and channels.

**Works with Sherlock OnDemand** — No port forwarding needed!

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CONFIGURATION - Edit these paths for your setup
# ═══════════════════════════════════════════════════════════════════════════════

PAIRS_DIR = "core_patches_npy"  # Directory with core_*_HE.npy and core_*_ORION.npy
CHECKPOINT_PATH = ""  # Path to model checkpoint (leave empty for view-only mode)
SCALER_PATH = ""  # Path to orion_scaler.json (optional if in checkpoint)

# Inference settings (only used if checkpoint provided)
PATCH_SIZE = 224
STRIDE = 112
DEVICE = "cpu"  # "cpu", "cuda", or "cuda:0"

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Imports and Setup
# ═══════════════════════════════════════════════════════════════════════════════

import json
from pathlib import Path
from typing import Optional, List, Dict, Tuple

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from IPython.display import display, clear_output
import ipywidgets as widgets

# Optional torch imports
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torchvision.transforms as T
    HAS_TORCH = True
except ImportError:
    HAS_TORCH = False
    print("⚠️ PyTorch not available - running in view-only mode")

try:
    import timm
    HAS_TIMM = True
except ImportError:
    HAS_TIMM = False

print(f"✅ Imports complete | PyTorch: {HAS_TORCH} | timm: {HAS_TIMM}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Marker Names and Colormaps
# ═══════════════════════════════════════════════════════════════════════════════

MARKER_NAMES = [
    "DAPI", "CD3", "CD31", "CD45", "CD34",
    "FoxP3", "CD56", "CD8", "CD11c", "PARP1",
    "CD206", "CD7", "HLA-DR", "Ki67", "Podoplanin",
    "CD4", "CA9", "Vimentin", "PD1", "CD11b",
]

COLORMAPS = [
    "viridis", "magma", "plasma", "inferno", "cividis",
    "Greens", "Blues", "Reds", "Purples", "Oranges",
    "hot", "turbo", "coolwarm", "gray", "bone"
]

N_CHANNELS = len(MARKER_NAMES)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Utility Functions
# ═══════════════════════════════════════════════════════════════════════════════

def _np_to_float01(a: np.ndarray) -> np.ndarray:
    """Normalize array to [0, 1] float32."""
    if a.dtype == np.uint8:
        a = a.astype(np.float32) / 255.0
    elif a.dtype in (np.uint16, np.int16):
        a = a.astype(np.float32)
        if a.max(initial=0.0) > 1.5:
            a = a / (np.percentile(a, 99.9) + 1e-6)
    elif a.dtype != np.float32:
        a = a.astype(np.float32)
    if a.max(initial=0.0) > 1.5:
        a = a / 255.0
    return a


def discover_basenames(pairs_dir: str) -> List[str]:
    """Find all core_*_HE.npy files with matching ORION."""
    d = Path(pairs_dir)
    out = []
    for hef in sorted(d.glob("core_*_HE.npy")):
        base = hef.stem.replace("_HE", "")
        if (d / f"{base}_ORION.npy").exists():
            out.append(base)
    return out


class QuantileScaler:
    """Global per-channel quantile normalization."""
    def __init__(self, q_low=1.0, q_high=99.5, C=20):
        self.q_low = q_low
        self.q_high = q_high
        self.qlo = np.zeros(C, dtype=np.float32)
        self.qhi = np.ones(C, dtype=np.float32)
        self.C = C

    @classmethod
    def from_dict(cls, d: Dict):
        obj = cls(d.get("q_low", 1.0), d.get("q_high", 99.5), d.get("C", 20))
        obj.qlo = np.array(d["qlo"], dtype=np.float32)
        obj.qhi = np.array(d["qhi"], dtype=np.float32)
        return obj

    @classmethod
    def load(cls, path: Path):
        return cls.from_dict(json.loads(path.read_text()))

    def scale_to_log(self, orion: np.ndarray) -> np.ndarray:
        C = self.C
        out = np.zeros_like(orion, dtype=np.float32)
        for c in range(C):
            x = (orion[..., c] - self.qlo[c]) / (self.qhi[c] - self.qlo[c] + 1e-6)
            x = np.clip(x, 0, None)
            out[..., c] = np.log1p(x)
        return out

    def inverse_log(self, log_data: np.ndarray) -> np.ndarray:
        return np.expm1(log_data)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Model Definition (for inference)
# ═══════════════════════════════════════════════════════════════════════════════

if HAS_TORCH and HAS_TIMM:
    class SwinUNet(nn.Module):
        def __init__(self, out_ch: int = 20, base_ch: int = 192, softplus_beta: float = 1.0):
            super().__init__()
            self.enc = timm.create_model(
                'swin_tiny_patch4_window7_224', pretrained=False, features_only=True, out_indices=(0,1,2,3)
            )
            enc_chs = self.enc.feature_info.channels()
            self.lats = nn.ModuleList([nn.Conv2d(c, base_ch, 1) for c in enc_chs])
            self.smooth3 = nn.Sequential(nn.Conv2d(base_ch, base_ch, 3, padding=1), nn.ReLU(inplace=True))
            self.smooth2 = nn.Sequential(nn.Conv2d(base_ch, base_ch, 3, padding=1), nn.ReLU(inplace=True))
            self.smooth1 = nn.Sequential(nn.Conv2d(base_ch, base_ch, 3, padding=1), nn.ReLU(inplace=True))
            self.smooth0 = nn.Sequential(nn.Conv2d(base_ch, base_ch//2, 3, padding=1), nn.ReLU(inplace=True))
            self.out = nn.Conv2d(base_ch//2, out_ch, 1)
            self.softplus = nn.Softplus(beta=softplus_beta)

        def forward(self, x):
            feats = self.enc(x)
            feats = [f.permute(0, 3, 1, 2) for f in feats]
            f3 = self.lats[3](feats[3])
            f2 = self._upsum(f3, self.lats[2](feats[2]))
            f2 = self.smooth3(f2)
            f1 = self._upsum(f2, self.lats[1](feats[1]))
            f1 = self.smooth2(f1)
            f0 = self._upsum(f1, self.lats[0](feats[0]))
            f0 = self.smooth1(f0)
            up = F.interpolate(f0, size=x.shape[-2:], mode='bilinear', align_corners=False)
            up = self.smooth0(up)
            y = self.out(up)
            y = self.softplus(y)
            return y

        @staticmethod
        def _up(x, size_hw):
            return F.interpolate(x, size=size_hw, mode='bilinear', align_corners=False)

        def _upsum(self, x_small, x_skip):
            x_up = self._up(x_small, x_skip.shape[-2:])
            return x_up + x_skip

    @torch.no_grad()
    def slide_reconstruct(model, he_img, tf_eval, ps=224, stride=112, device="cpu"):
        H, W, _ = he_img.shape
        out_accum = None
        weight = None
        for y in range(0, max(1, H - ps) + 1, stride):
            for x in range(0, max(1, W - ps) + 1, stride):
                he_crop = (he_img[y:y+ps, x:x+ps, :] * 255).astype(np.uint8)
                he_t = tf_eval(he_crop).unsqueeze(0).to(device)
                pred_log = model(he_t).detach().cpu().numpy()[0]
                if out_accum is None:
                    C = pred_log.shape[0]
                    out_accum = np.zeros((C, H, W), dtype=np.float32)
                    weight = np.zeros((1, H, W), dtype=np.float32)
                y2 = min(H, y + ps)
                x2 = min(W, x + ps)
                ph, pw = y2 - y, x2 - x
                out_accum[:, y:y2, x:x2] += pred_log[:, :ph, :pw]
                weight[:, y:y2, x:x2] += 1.0
        return out_accum / np.clip(weight, 1e-6, None)

    print("✅ Model class defined")
else:
    print("ℹ️ Model not available - view-only mode")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Load Data and Model
# ═══════════════════════════════════════════════════════════════════════════════

# Discover cores
pairs_dir = Path(PAIRS_DIR)
if not pairs_dir.exists():
    raise FileNotFoundError(f"❌ Data directory not found: {PAIRS_DIR}")

BASENAMES = discover_basenames(PAIRS_DIR)
print(f"✅ Found {len(BASENAMES)} paired cores")

# Load scaler
scaler = None
model = None
tf_eval = None

if CHECKPOINT_PATH and Path(CHECKPOINT_PATH).exists() and HAS_TORCH and HAS_TIMM:
    print(f"Loading model from {CHECKPOINT_PATH}...")
    ckpt = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
    model = SwinUNet(out_ch=20, base_ch=192, softplus_beta=1.0)
    model.load_state_dict(ckpt["model"])
    model.to(DEVICE)
    model.eval()
    if "scaler" in ckpt:
        scaler = QuantileScaler.from_dict(ckpt["scaler"])
    tf_eval = T.Compose([
        T.ToPILImage(),
        T.Resize(PATCH_SIZE, antialias=True),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    print("✅ Model loaded!")

if scaler is None and SCALER_PATH and Path(SCALER_PATH).exists():
    scaler = QuantileScaler.load(Path(SCALER_PATH))
    print("✅ Scaler loaded from file")

if scaler is None:
    scaler = QuantileScaler(q_low=1.0, q_high=99.5, C=N_CHANNELS)
    print("ℹ️ Using default scaler (no quantile normalization)")

HAS_MODEL = model is not None
print(f"\n🎯 Mode: {'Prediction + GT' if HAS_MODEL else 'View-only (GT only)'}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Data Loading Functions with Caching
# ═══════════════════════════════════════════════════════════════════════════════

_cache = {}

def load_core(basename: str):
    """Load H&E and ORION for a core (with caching)."""
    if basename in _cache:
        return _cache[basename]
    
    he = np.load(pairs_dir / f"{basename}_HE.npy")
    orion = np.load(pairs_dir / f"{basename}_ORION.npy")
    
    if orion.ndim == 3 and orion.shape[0] == N_CHANNELS:
        orion = np.transpose(orion, (1, 2, 0))
    
    he = _np_to_float01(he)
    orion = _np_to_float01(orion)
    
    _cache[basename] = (he, orion)
    return he, orion


def get_prediction(basename: str, he: np.ndarray):
    """Get model prediction (with caching)."""
    cache_key = f"{basename}_pred"
    if cache_key in _cache:
        return _cache[cache_key]
    
    if not HAS_MODEL:
        return None
    
    pred_log = slide_reconstruct(model, he, tf_eval, ps=PATCH_SIZE, stride=STRIDE, device=DEVICE)
    _cache[cache_key] = pred_log
    return pred_log


def compute_metrics(gt, pred):
    """Compute comparison metrics."""
    gt_flat = gt.flatten()
    pred_flat = pred.flatten()
    mse = np.mean((gt_flat - pred_flat) ** 2)
    mae = np.mean(np.abs(gt_flat - pred_flat))
    if gt_flat.std() > 1e-8 and pred_flat.std() > 1e-8:
        corr = np.corrcoef(gt_flat, pred_flat)[0, 1]
    else:
        corr = 0.0
    return {"MSE": mse, "MAE": mae, "Pearson r": corr}

print("✅ Loading functions ready")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎮 INTERACTIVE VIEWER - Run this cell!
# ═══════════════════════════════════════════════════════════════════════════════

# Create widgets
core_dropdown = widgets.Dropdown(
    options=[(f"{b} ({i+1}/{len(BASENAMES)})", b) for i, b in enumerate(BASENAMES)],
    value=BASENAMES[0],
    description='Core:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='300px')
)

channel_dropdown = widgets.Dropdown(
    options=[(f"Ch {i}: {MARKER_NAMES[i]}", i) for i in range(N_CHANNELS)],
    value=0,
    description='Channel:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='250px')
)

cmap_dropdown = widgets.Dropdown(
    options=COLORMAPS,
    value='viridis',
    description='Colormap:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='180px')
)

vmin_slider = widgets.FloatSlider(
    value=0.0, min=0.0, max=1.0, step=0.01,
    description='V Min:',
    style={'description_width': '50px'},
    layout=widgets.Layout(width='250px')
)

vmax_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=2.0, step=0.01,
    description='V Max:',
    style={'description_width': '50px'},
    layout=widgets.Layout(width='250px')
)

show_hist_check = widgets.Checkbox(
    value=False,
    description='Show Histogram',
    layout=widgets.Layout(width='150px')
)

prev_button = widgets.Button(description='◀ Prev', layout=widgets.Layout(width='80px'))
next_button = widgets.Button(description='Next ▶', layout=widgets.Layout(width='80px'))

output = widgets.Output()


def update_view(*args):
    """Update the visualization."""
    with output:
        clear_output(wait=True)
        
        basename = core_dropdown.value
        channel = channel_dropdown.value
        cmap = cmap_dropdown.value
        vmin = vmin_slider.value
        vmax = vmax_slider.value
        
        # Load data
        print(f"Loading {basename}...", end=" ")
        he, orion = load_core(basename)
        gt_log = scaler.scale_to_log(orion)
        print("✓")
        
        # Get prediction if available
        pred_log = None
        if HAS_MODEL:
            print("Running inference...", end=" ")
            pred_log = get_prediction(basename, he)
            print("✓")
        
        # Create figure
        n_cols = 3 if pred_log is not None else 2
        fig_width = 5 * n_cols
        fig, axes = plt.subplots(1, n_cols, figsize=(fig_width, 5))
        if n_cols == 2:
            axes = list(axes)
        
        # H&E
        axes[0].imshow(he)
        axes[0].set_title(f"H&E — {basename}", fontsize=10)
        axes[0].axis('off')
        
        # Ground Truth
        gt_ch = scaler.inverse_log(gt_log[..., channel])
        im_gt = axes[1].imshow(gt_ch, cmap=cmap, vmin=vmin, vmax=vmax)
        axes[1].set_title(f"Ground Truth — {MARKER_NAMES[channel]}", fontsize=10)
        axes[1].axis('off')
        plt.colorbar(im_gt, ax=axes[1], fraction=0.046, pad=0.04)
        
        # Prediction
        if pred_log is not None:
            pred_ch = scaler.inverse_log(pred_log[channel])
            im_pred = axes[2].imshow(pred_ch, cmap=cmap, vmin=vmin, vmax=vmax)
            axes[2].set_title(f"Prediction — {MARKER_NAMES[channel]}", fontsize=10)
            axes[2].axis('off')
            plt.colorbar(im_pred, ax=axes[2], fraction=0.046, pad=0.04)
            
            # Metrics
            metrics = compute_metrics(gt_ch, pred_ch)
            metrics_str = " | ".join([f"{k}: {v:.4f}" for k, v in metrics.items()])
            fig.suptitle(metrics_str, fontsize=9, y=0.02)
        
        plt.tight_layout()
        plt.show()
        
        # Histogram
        if show_hist_check.value:
            fig_hist, ax_hist = plt.subplots(figsize=(10, 3))
            ax_hist.hist(gt_ch.flatten(), bins=100, alpha=0.7, label="Ground Truth", color="#00d4aa")
            if pred_log is not None:
                ax_hist.hist(pred_ch.flatten(), bins=100, alpha=0.7, label="Prediction", color="#ff6b6b")
            ax_hist.set_xlabel("Intensity")
            ax_hist.set_ylabel("Frequency")
            ax_hist.set_title(f"Intensity Distribution — {MARKER_NAMES[channel]}")
            ax_hist.legend()
            ax_hist.set_xlim(0, max(vmax, 1.0))
            plt.tight_layout()
            plt.show()


def on_prev_click(b):
    idx = BASENAMES.index(core_dropdown.value)
    if idx > 0:
        core_dropdown.value = BASENAMES[idx - 1]


def on_next_click(b):
    idx = BASENAMES.index(core_dropdown.value)
    if idx < len(BASENAMES) - 1:
        core_dropdown.value = BASENAMES[idx + 1]


# Connect widgets
core_dropdown.observe(update_view, names='value')
channel_dropdown.observe(update_view, names='value')
cmap_dropdown.observe(update_view, names='value')
vmin_slider.observe(update_view, names='value')
vmax_slider.observe(update_view, names='value')
show_hist_check.observe(update_view, names='value')
prev_button.on_click(on_prev_click)
next_button.on_click(on_next_click)

# Layout
controls_row1 = widgets.HBox([core_dropdown, channel_dropdown, cmap_dropdown])
controls_row2 = widgets.HBox([vmin_slider, vmax_slider, show_hist_check])
nav_row = widgets.HBox([prev_button, next_button])

display(widgets.VBox([
    widgets.HTML("<h2>🔬 HEXIF Core Viewer</h2>"),
    controls_row1,
    controls_row2,
    nav_row,
    output
]))

# Initial render
update_view()

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Multi-Channel Overview (Run to see all channels at once)
# ═══════════════════════════════════════════════════════════════════════════════

def show_all_channels(basename: str, cmap: str = 'viridis'):
    """Display all channels for a single core."""
    he, orion = load_core(basename)
    gt_log = scaler.scale_to_log(orion)
    
    n_cols = 5
    n_rows = (N_CHANNELS + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 3 * n_rows))
    axes = axes.flatten()
    
    for ch in range(N_CHANNELS):
        ch_data = scaler.inverse_log(gt_log[..., ch])
        axes[ch].imshow(ch_data, cmap=cmap, vmin=0, vmax=0.5)
        axes[ch].set_title(f"{MARKER_NAMES[ch]}", fontsize=9)
        axes[ch].axis('off')
    
    for i in range(N_CHANNELS, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(f"All Channels — {basename}", fontsize=12)
    plt.tight_layout()
    plt.show()

# Uncomment and run to see all channels for the first core:
# show_all_channels(BASENAMES[0], cmap='magma')

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Batch Export (Save comparison images for all cores)
# ═══════════════════════════════════════════════════════════════════════════════

def batch_export(output_dir: str, channels: list = [0, 1, 2, 3, 4], cmap: str = 'viridis'):
    """Export comparison images for all cores."""
    out_path = Path(output_dir)
    out_path.mkdir(parents=True, exist_ok=True)
    
    for i, basename in enumerate(BASENAMES):
        print(f"Processing {basename} ({i+1}/{len(BASENAMES)})...", end=" ")
        
        he, orion = load_core(basename)
        gt_log = scaler.scale_to_log(orion)
        pred_log = get_prediction(basename, he) if HAS_MODEL else None
        
        n_cols = 3 if pred_log is not None else 2
        for ch in channels:
            fig, axes = plt.subplots(1, n_cols, figsize=(4*n_cols, 4))
            
            axes[0].imshow(he)
            axes[0].set_title("H&E")
            axes[0].axis('off')
            
            gt_ch = scaler.inverse_log(gt_log[..., ch])
            axes[1].imshow(gt_ch, cmap=cmap, vmin=0, vmax=0.5)
            axes[1].set_title(f"GT: {MARKER_NAMES[ch]}")
            axes[1].axis('off')
            
            if pred_log is not None:
                pred_ch = scaler.inverse_log(pred_log[ch])
                axes[2].imshow(pred_ch, cmap=cmap, vmin=0, vmax=0.5)
                axes[2].set_title(f"Pred: {MARKER_NAMES[ch]}")
                axes[2].axis('off')
            
            plt.tight_layout()
            fig.savefig(out_path / f"{basename}_ch{ch:02d}_{MARKER_NAMES[ch]}.png", dpi=150, bbox_inches='tight')
            plt.close(fig)
        
        print("✓")
    
    print(f"\n✅ Exported to {out_path}")

# Uncomment to export:
# batch_export("viewer_exports", channels=[0, 1, 5, 7, 15])